# 📊 Первичная аналитика: task_1_candles.csv

Этот ноутбук содержит первичную аналитику данных о свечах (OHLCV) для быстрого онбординга.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10


## 1. Загрузка данных


In [ ]:
# Загрузка данных
df = pd.read_csv('../data/raw/task_1_candles.csv')

# Конвертация даты
df['begin'] = pd.to_datetime(df['begin'])

# Сортировка
df = df.sort_values(['ticker', 'begin']).reset_index(drop=True)

print(f"Размер датасета: {df.shape}")
print(f"Период: {df['begin'].min()} - {df['begin'].max()}")
df.head(10)


## 2. Общая информация о данных


In [ ]:
# Информация о типах данных
print("=== Типы данных ===")
print(df.dtypes)
print("\n=== Пропущенные значения ===")
print(df.isnull().sum())
print("\n=== Базовая статистика ===")
df.describe()


## 3. Анализ тикеров


In [ ]:
# Список тикеров
tickers = df['ticker'].unique()
print(f"Количество уникальных тикеров: {len(tickers)}")
print(f"\nТикеры: {sorted(tickers)}")


In [ ]:
# Количество записей по каждому тикеру
ticker_counts = df['ticker'].value_counts().sort_index()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Барплот
ticker_counts.plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Количество торговых дней по тикерам', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Тикер')
axes[0].set_ylabel('Количество записей')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Таблица со статистикой
axes[1].axis('off')
table_data = []
for ticker in sorted(tickers):
    ticker_df = df[df['ticker'] == ticker]
    table_data.append([
        ticker,
        len(ticker_df),
        ticker_df['begin'].min().strftime('%Y-%m-%d'),
        ticker_df['begin'].max().strftime('%Y-%m-%d')
    ])

table = axes[1].table(cellText=table_data,
                      colLabels=['Тикер', 'Записей', 'Начало', 'Конец'],
                      cellLoc='center', loc='center',
                      colColours=['lightgray']*4)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)
axes[1].set_title('Временные периоды по тикерам', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("\nСтатистика по тикерам:")
print(ticker_counts.describe())


## 4. Временной анализ


In [ ]:
# Анализ временных периодов
print(f"Общий период данных: {df['begin'].min().date()} - {df['begin'].max().date()}")
print(f"Длительность: {(df['begin'].max() - df['begin'].min()).days} дней")

# Количество записей по времени
df['year'] = df['begin'].dt.year
df['month'] = df['begin'].dt.month
df['year_month'] = df['begin'].dt.to_period('M')

# Графики временных рядов
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Количество записей по месяцам
monthly_counts = df.groupby('year_month').size()
monthly_counts.plot(ax=axes[0], marker='o', linewidth=2, color='darkblue')
axes[0].set_title('Количество записей по месяцам', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Период')
axes[0].set_ylabel('Количество записей')
axes[0].grid(True, alpha=0.3)

# Количество уникальных тикеров по месяцам
monthly_tickers = df.groupby('year_month')['ticker'].nunique()
monthly_tickers.plot(ax=axes[1], marker='s', linewidth=2, color='darkgreen')
axes[1].set_title('Количество активных тикеров по месяцам', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Период')
axes[1].set_ylabel('Количество тикеров')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. Анализ цен


In [ ]:
# Статистика по ценам
print("=== Статистика по ценам ===")
print(df[['open', 'close', 'high', 'low']].describe())

# Графики цен для каждого тикера
n_tickers = len(tickers)
n_cols = 3
n_rows = (n_tickers + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows*4))
axes = axes.flatten() if n_tickers > 1 else [axes]

for idx, ticker in enumerate(sorted(tickers)):
    ticker_df = df[df['ticker'] == ticker].sort_values('begin')
    ax = axes[idx]

    ax.plot(ticker_df['begin'], ticker_df['close'], label='Close', linewidth=2, alpha=0.8)
    ax.fill_between(ticker_df['begin'], ticker_df['low'], ticker_df['high'],
                     alpha=0.2, label='High-Low Range')

    ax.set_title(f'{ticker} - Цена закрытия и диапазон High-Low', fontweight='bold')
    ax.set_xlabel('Дата')
    ax.set_ylabel('Цена')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.tick_params(axis='x', rotation=45)

# Скрыть лишние subplot'ы
for idx in range(n_tickers, len(axes)):
    axes[idx].set_visible(False)

plt.tight_layout()
plt.show()


## 6. Анализ доходности


In [ ]:
# Расчет дневной доходности
df['daily_return'] = df.groupby('ticker')['close'].pct_change()
df['intraday_return'] = (df['close'] - df['open']) / df['open']

print("=== Статистика доходности ===")
print(df[['daily_return', 'intraday_return']].describe())


In [ ]:
# Распределение доходностей
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Дневная доходность - гистограмма
df['daily_return'].dropna().hist(bins=100, ax=axes[0, 0], edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение дневной доходности (все тикеры)', fontweight='bold')
axes[0, 0].set_xlabel('Доходность')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].axvline(0, color='red', linestyle='--', linewidth=2, label='0')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Внутридневная доходность - гистограмма
df['intraday_return'].dropna().hist(bins=100, ax=axes[0, 1], edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Распределение внутридневной доходности (все тикеры)', fontweight='bold')
axes[0, 1].set_xlabel('Доходность')
axes[0, 1].set_ylabel('Частота')
axes[0, 1].axvline(0, color='red', linestyle='--', linewidth=2, label='0')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Boxplot по тикерам - дневная доходность
df.boxplot(column='daily_return', by='ticker', ax=axes[1, 0])
axes[1, 0].set_title('Распределение дневной доходности по тикерам', fontweight='bold')
axes[1, 0].set_xlabel('Тикер')
axes[1, 0].set_ylabel('Доходность')
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=1)
plt.sca(axes[1, 0])
plt.xticks(rotation=45)

# Boxplot по тикерам - внутридневная доходность
df.boxplot(column='intraday_return', by='ticker', ax=axes[1, 1])
axes[1, 1].set_title('Распределение внутридневной доходности по тикерам', fontweight='bold')
axes[1, 1].set_xlabel('Тикер')
axes[1, 1].set_ylabel('Доходность')
axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=1)
plt.sca(axes[1, 1])
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
# Статистика доходности по тикерам
returns_stats = df.groupby('ticker')['daily_return'].agg([
    ('mean', 'mean'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max'),
    ('median', 'median')
]).round(4)

returns_stats.columns = ['Средняя', 'Ст. откл.', 'Минимум', 'Максимум', 'Медиана']
print("\n=== Статистика дневной доходности по тикерам ===")
print(returns_stats)

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

returns_stats['Средняя'].plot(kind='bar', ax=axes[0], color='green', alpha=0.7)
axes[0].set_title('Средняя дневная доходность по тикерам', fontweight='bold')
axes[0].set_ylabel('Доходность')
axes[0].axhline(0, color='red', linestyle='--', linewidth=1)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

returns_stats['Ст. откл.'].plot(kind='bar', ax=axes[1], color='red', alpha=0.7)
axes[1].set_title('Волатильность (стандартное отклонение) по тикерам', fontweight='bold')
axes[1].set_ylabel('Ст. отклонение')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 7. Анализ объемов торгов


In [ ]:
# Статистика по объемам
print("=== Статистика по объемам торгов ===")
print(df['volume'].describe())

volume_stats = df.groupby('ticker')['volume'].agg([
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max')
]).round(0)

print("\n=== Объемы торгов по тикерам ===")
print(volume_stats)


In [ ]:
# Визуализация объемов
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Распределение объемов
df['volume'].hist(bins=100, ax=axes[0, 0], edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение объемов торгов (все тикеры)', fontweight='bold')
axes[0, 0].set_xlabel('Объем')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

# Логарифм объемов (для лучшей визуализации)
df['log_volume'] = np.log1p(df['volume'])
df['log_volume'].hist(bins=100, ax=axes[0, 1], edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].set_title('Распределение log(объемов)', fontweight='bold')
axes[0, 1].set_xlabel('log(Объем)')
axes[0, 1].set_ylabel('Частота')
axes[0, 1].grid(True, alpha=0.3)

# Средний объем по тикерам
volume_stats['mean'].plot(kind='bar', ax=axes[1, 0], color='steelblue', alpha=0.7)
axes[1, 0].set_title('Средний объем торгов по тикерам', fontweight='bold')
axes[1, 0].set_ylabel('Объем')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(True, alpha=0.3)

# Boxplot объемов по тикерам
df.boxplot(column='volume', by='ticker', ax=axes[1, 1])
axes[1, 1].set_title('Распределение объемов по тикерам', fontweight='bold')
axes[1, 1].set_xlabel('Тикер')
axes[1, 1].set_ylabel('Объем')
plt.sca(axes[1, 1])
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


## 8. Анализ волатильности


In [ ]:
# Расчет различных мер волатильности
df['price_range'] = df['high'] - df['low']
df['price_range_pct'] = (df['high'] - df['low']) / df['low'] * 100

# Скользящая волатильность (окно 20 дней)
df['rolling_volatility'] = df.groupby('ticker')['daily_return'].transform(
    lambda x: x.rolling(window=20, min_periods=1).std()
)

print("=== Статистика волатильности ===")
print(df[['price_range', 'price_range_pct', 'rolling_volatility']].describe())


In [ ]:
# Визуализация волатильности
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Средняя волатильность по тикерам
volatility_by_ticker = df.groupby('ticker')['rolling_volatility'].mean().sort_values(ascending=False)
volatility_by_ticker.plot(kind='bar', ax=axes[0], color='purple', alpha=0.7)
axes[0].set_title('Средняя волатильность (20-дневная) по тикерам', fontweight='bold')
axes[0].set_ylabel('Волатильность')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Временной ряд волатильности для всех тикеров
for ticker in sorted(tickers):
    ticker_df = df[df['ticker'] == ticker].sort_values('begin')
    axes[1].plot(ticker_df['begin'], ticker_df['rolling_volatility'],
                label=ticker, alpha=0.7, linewidth=1.5)

axes[1].set_title('Временной ряд волатильности (20-дневная) по тикерам', fontweight='bold')
axes[1].set_xlabel('Дата')
axes[1].set_ylabel('Волатильность')
axes[1].legend(loc='best', ncol=3)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 9. Корреляционный анализ


In [ ]:
# Создаем pivot таблицу с ценами закрытия
pivot_close = df.pivot_table(index='begin', columns='ticker', values='close', aggfunc='mean')

# Расчет корреляций
correlation_matrix = pivot_close.corr()

# Визуализация
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Корреляционная матрица цен закрытия', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n=== Корреляционная матрица ===")
print(correlation_matrix)


In [ ]:
# Корреляция доходностей
pivot_returns = df.pivot_table(index='begin', columns='ticker', values='daily_return', aggfunc='mean')
returns_correlation = pivot_returns.corr()

fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(returns_correlation, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
ax.set_title('Корреляционная матрица доходностей', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n=== Корреляционная матрица доходностей ===")
print(returns_correlation)


## 10. Итоговая сводка


In [ ]:
print("="*80)
print("ИТОГОВАЯ СВОДКА ПО ДАННЫМ")
print("="*80)

print(f"\n📊 Общая информация:")
print(f"  - Всего записей: {len(df):,}")
print(f"  - Количество тикеров: {len(tickers)}")
print(f"  - Период данных: {df['begin'].min().date()} - {df['begin'].max().date()}")
print(f"  - Длительность: {(df['begin'].max() - df['begin'].min()).days} дней")

print(f"\n📈 Тикеры:")
print(f"  {', '.join(sorted(tickers))}")

print(f"\n💰 Ценовая статистика:")
print(f"  - Минимальная цена: {df['low'].min():.2f}")
print(f"  - Максимальная цена: {df['high'].max():.2f}")
print(f"  - Средняя цена закрытия: {df['close'].mean():.2f}")

print(f"\n📊 Доходность:")
print(f"  - Средняя дневная доходность: {df['daily_return'].mean()*100:.4f}%")
print(f"  - Стандартное отклонение: {df['daily_return'].std()*100:.4f}%")
print(f"  - Минимальная доходность: {df['daily_return'].min()*100:.2f}%")
print(f"  - Максимальная доходность: {df['daily_return'].max()*100:.2f}%")

print(f"\n📦 Объемы торгов:")
print(f"  - Средний объем: {df['volume'].mean():,.0f}")
print(f"  - Медианный объем: {df['volume'].median():,.0f}")
print(f"  - Максимальный объем: {df['volume'].max():,.0f}")

print(f"\n⚡ Волатильность:")
for ticker in sorted(tickers):
    ticker_vol = df[df['ticker'] == ticker]['rolling_volatility'].mean()
    print(f"  - {ticker}: {ticker_vol*100:.4f}%")

print(f"\n✅ Качество данных:")
print(f"  - Пропущенные значения: {df.isnull().sum().sum()}")
print(f"  - Дубликаты: {df.duplicated().sum()}")

print("\n" + "="*80)
